In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

imgfile_dir = "./archive/garbage classification/Garbage classification/"

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-12-22 11:13:24.116437: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/api/keras/optimizers


AlreadyExistsError: Another metric with the same name already exists.

In [ ]:
classes = os.listdir(imgfile_dir)
classes

['cardboard', 'trash', 'glass', 'paper', 'plastic', 'metal']

In [ ]:
SIZE = 128

# configuracion de entrenamiento
entrenamiento_datagen = ImageDataGenerator(rescale = 1./255)

# generador
generador_entrenamiento = entrenamiento_datagen.flow_from_directory(
    imgfile_dir,
    target_size=(SIZE,SIZE),
    batch_size=128,
    class_mode='binary'
)

Found 2527 images belonging to 6 classes.


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.5),
    ]
)

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image SIZExSIZE with 3 bytes color
    tf.keras.layers.Dense(50, activation='relu', input_shape=(128, 128, 3)),
    # This is the first convolution
    tf.keras.layers.Conv2D(50, (3,3), activation='relu'),
    tf.keras.layers.SpatialDropout2D(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
     # This is the first convolution
    tf.keras.layers.Conv2D(50, (3,3), activation='relu'),
    tf.keras.layers.SpatialDropout2D(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128, 128, 50)      200       
                                                                 
 conv2d_6 (Conv2D)           (None, 126, 126, 50)      22550     
                                                                 
 spatial_dropout2d_6 (Spatia  (None, 126, 126, 50)     0         
 lDropout2D)                                                     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 50)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 50)        22550     
                                                                 
 spatial_dropout2d_7 (Spatia  (None, 61, 61, 50)      

In [ ]:
from tensorflow.keras.optimizers import Adagrad

model.compile(loss='binary_crossentropy',
              optimizer=Adagrad(learning_rate=0.1),
              metrics=['accuracy'])

history = model.fit(
    generador_entrenamiento,
    steps_per_epoch=20,  
    epochs=30,
    verbose=1
)

ImportError: Image transformations require SciPy. Install SciPy.

In [ ]:
score = model.evaluate_generator (generador_entrenamiento,verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Test loss: -1.724327318172268e+20
Test accuracy: 0.19825880229473114
